In [76]:
import pandas as pd

# Sample DataFrame
data = {

    "issue_area": [
        "Login and Account", "Cancellations and returns", "Login and Account", "Order", "Order"
    ],
    "issue_category": [
        "Mobile Number and Email Verification", "Pickup and Shipping", "Replacement and Return Process",
        "Login Issues and Error Messages", "Order Delivery Issues"
    ],
    "issue_sub_category": [
        "Verification requirement for mobile number or email",
        "Reasons for being asked to ship the item",
        "Inability to click the 'Cancel' button",
        "Error message regarding exceeded attempts to enter password",
        "Delivery not attempted again"
    ],
    "issue_category_sub_category": [
        "Mobile Number and Email Verification -> Verification requirement for mobile number or email",
        "Pickup and Shipping -> Reasons for being asked to ship the item",
        "Replacement and Return Process -> Inability to click the 'Cancel' button",
        "Login Issues and Error Messages -> Error message regarding exceeded attempts to enter password",
        "Order Delivery Issues -> Delivery not attempted again"
    ],
    "customer_sentiment": ["neutral", "neutral", "neutral", "neutral", "negative"],
    "product_category": ["Appliances", "Appliances", "Appliances", "Appliances", "Electronics"],
    "product_sub_category": [
        "Oven Toaster Grills (OTG)", "Computer Monitor", "Juicer/Mixer/Grinder", "Water Purifier", "Bp Monitor"
    ],
    "issue_complexity": ["medium", "less", "medium", "less", "medium"],
    "agent_experience_level": ["junior", "junior", "experienced", "inexperienced", "experienced"],
    "agent_experience_level_desc": [
        "handles customer inquiries independently, possesses good knowledge",
        "handles customer inquiries independently, possesses good knowledge",
        "confidently handles complex customer issues, experienced",
        "may struggle with ambiguous queries, rely on confirmation",
        "confidently handles complex customer issues, experienced"
    ],
    "conversation": [
        "Agent: Thank you for calling BrownBox Customer Support...",
        "Agent: Thank you for calling BrownBox Customer Support...",
        "Agent: Thank you for calling BrownBox Customer Support...",
        "Customer: Hi, I am facing an issue while logging in...",
        "Agent: Thank you for calling BrownBox Customer Support..."
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to Parquet
df.to_parquet("sample.parquet", engine="pyarrow", index=False)  # or use engine="fastparquet"


In [77]:
df.head()

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Appliances,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
2,Login and Account,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Order,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...","Customer: Hi, I am facing an issue while loggi..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...


In [78]:
!pip install transformers
from transformers import pipeline

# Load the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# Assuming 'df' DataFrame has a column named 'text' containing the text for analysis
# Replace 'text' with the actual column name if it's different

# Process the first 10 rows for demonstration.
for index, row in df.head(10).iterrows(): # Changed 'data' to 'df'
    text = row['issue_area']
    try:
        result = classifier(text)
        print(f"Text: {text[:50]}...") # Print first 50 characters of text
        print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']}")
    except Exception as e:
        print(f"Error analyzing text '{text[:50]}...': {e}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Text: Login and Account...
Sentiment: NEGATIVE, Score: 0.9505677223205566
Text: Cancellations and returns...
Sentiment: NEGATIVE, Score: 0.9977436065673828
Text: Login and Account...
Sentiment: NEGATIVE, Score: 0.9505677223205566
Text: Order...
Sentiment: POSITIVE, Score: 0.9820718169212341
Text: Order...
Sentiment: POSITIVE, Score: 0.9820718169212341


In [79]:
from textblob import TextBlob
import pandas as pd # Import pandas

# ... (your existing code) ...

# Create DataFrame before applying sentiment analysis
df = pd.DataFrame(data) # Create DataFrame from the dictionary

# Apply sentiment analysis to the 'conversation' column of the DataFrame
df['predicted_sentiment'] = df['conversation'].apply(analyze_sentiment)

# Display the results
output = df[['conversation', 'customer_sentiment', 'predicted_sentiment']]
print(output)

                                        conversation customer_sentiment  \
0  Agent: Thank you for calling BrownBox Customer...            neutral   
1  Agent: Thank you for calling BrownBox Customer...            neutral   
2  Agent: Thank you for calling BrownBox Customer...            neutral   
3  Customer: Hi, I am facing an issue while loggi...            neutral   
4  Agent: Thank you for calling BrownBox Customer...           negative   

  predicted_sentiment  
0             neutral  
1             neutral  
2             neutral  
3             neutral  
4             neutral  


In [80]:
pip install vaderSentiment


In [81]:
import pandas as pd
positive_words = ['thank', 'great', 'good', 'excellent', 'happy', 'amazing']
negative_words = ['issue', 'problem', 'bad', 'poor', 'angry', 'not']

def keyword_sentiment(text):
    text = str(text).lower()
    pos_count = sum(word in text for word in positive_words)
    neg_count = sum(word in text for word in negative_words)
    if pos_count > neg_count:
        return 'positive'
    elif neg_count > pos_count:
        return 'negative'
    else:
        return 'neutral'

# Convert 'data' to a DataFrame
df = pd.DataFrame(data) # Creating a DataFrame from your existing 'data' dictionary

# Apply sentiment analysis to the 'conversation' column of the DataFrame
df['keyword_sentiment'] = df['conversation'].apply(keyword_sentiment) # Applying the function to the DataFrame

# Display the results
output = df[['conversation', 'customer_sentiment', 'keyword_sentiment']]
print(output.head())

                                        conversation customer_sentiment  \
0  Agent: Thank you for calling BrownBox Customer...            neutral   
1  Agent: Thank you for calling BrownBox Customer...            neutral   
2  Agent: Thank you for calling BrownBox Customer...            neutral   
3  Customer: Hi, I am facing an issue while loggi...            neutral   
4  Agent: Thank you for calling BrownBox Customer...           negative   

  keyword_sentiment  
0          positive  
1          positive  
2          positive  
3          negative  
4          positive  


In [82]:
# NAIVE BAYES CLASSIFIER FOR SENTIMENT ANALYSIS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Prepare the data
X = data['conversation']
y = data['customer_sentiment']

# Convert text to numerical features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X_vectorized = vectorizer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     neutral       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [83]:
import re
import pandas as pd # Import pandas

# Define aspect-specific keywords
aspect_keywords = {
    'agent': ['agent', 'support', 'customer service'],
    'product': ['product', 'item', 'quality', 'defect'],
    'delivery': ['delivery', 'shipping', 'time', 'delayed']
}

# Define positive and negative keywords
positive_words = ['thank', 'great', 'good', 'excellent', 'happy', 'amazing']
negative_words = ['issue', 'problem', 'bad', 'poor', 'angry', 'not']

# Function for aspect-based sentiment analysis
def aspect_sentiment_analysis(text):
    text = str(text).lower()
    results = {}
    for aspect, keywords in aspect_keywords.items():
        if any(re.search(rf'\b{keyword}\b', text) for keyword in keywords):
            pos_count = sum(word in text for word in positive_words)
            neg_count = sum(word in text for word in negative_words)
            if pos_count > neg_count:
                results[aspect] = 'positive'
            elif neg_count > pos_count:
                results[aspect] = 'negative'
            else:
                results[aspect] = 'neutral'
    return results

# Convert 'data' to a DataFrame before applying the function
df = pd.DataFrame(data)  # Create a DataFrame from the 'data' dictionary

# Apply the function to the 'conversation' column of the DataFrame
df['aspect_sentiment'] = df['conversation'].apply(aspect_sentiment_analysis)

# Print the results
print(df[['conversation', 'aspect_sentiment']].head())

                                        conversation       aspect_sentiment
0  Agent: Thank you for calling BrownBox Customer...  {'agent': 'positive'}
1  Agent: Thank you for calling BrownBox Customer...  {'agent': 'positive'}
2  Agent: Thank you for calling BrownBox Customer...  {'agent': 'positive'}
3  Customer: Hi, I am facing an issue while loggi...                     {}
4  Agent: Thank you for calling BrownBox Customer...  {'agent': 'positive'}


In [84]:
# Clustering-Based Sentiment Analysis
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd # Import pandas to convert data to a DataFrame

# Convert data to DataFrame
data = pd.DataFrame(data)  # Convert 'data' to a DataFrame if it's not already

# Convert conversations to TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(data['conversation'])

# Apply k-means clustering
num_clusters = 3  # For positive, neutral, negative
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(X_tfidf)

# Manually label clusters (based on inspection)
cluster_labels = {0: 'positive', 1: 'neutral', 2: 'negative'}
# Map cluster labels to cluster sentiment using pandas 'map' function
data['cluster_sentiment'] = data['cluster'].map(cluster_labels) # Using pandas 'map' function

# Display results
print(data[['conversation', 'cluster_sentiment']].head())

                                        conversation cluster_sentiment
0  Agent: Thank you for calling BrownBox Customer...          positive
1  Agent: Thank you for calling BrownBox Customer...          positive
2  Agent: Thank you for calling BrownBox Customer...          positive
3  Customer: Hi, I am facing an issue while loggi...           neutral
4  Agent: Thank you for calling BrownBox Customer...          positive


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [85]:
# Optional(only if emojis are present in the data)
import re

def emoji_sentiment(text):
    positive_emojis = [r':\)', r':D', r'😊', r'😍', r'👍']
    negative_emojis = [r':\(', r':/', r'😠', r'😢', r'👎']

    if any(re.search(emoji, text) for emoji in positive_emojis):
        return 'positive'
    elif any(re.search(emoji, text) for emoji in negative_emojis):
        return 'negative'
    else:
        return 'neutral'

data['emoji_sentiment'] = data['conversation'].apply(emoji_sentiment)

# Display results
print(data[['conversation', 'emoji_sentiment']].head())

                                        conversation emoji_sentiment
0  Agent: Thank you for calling BrownBox Customer...         neutral
1  Agent: Thank you for calling BrownBox Customer...         neutral
2  Agent: Thank you for calling BrownBox Customer...         neutral
3  Customer: Hi, I am facing an issue while loggi...         neutral
4  Agent: Thank you for calling BrownBox Customer...         neutral
